# IRWS Homework 03
*Sebastian Wagner*
## Exercise 1
### a)

In [86]:
from pprint import pprint
import numpy as np
import nltk
import string
import math

from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

defining the documents here and set the vocabulary

In [87]:
TOKENIZER = TreebankWordTokenizer()
LEMMATIZER = WordNetLemmatizer()


d1 = "Frodo and Sam were trembling in the darkness, surrounded in darkness by hundreds of blood-thirsty orc. Sam was certain these beasts were about to taste the scent of their flesh."
d2 = "The faceless black beast then stabbed Frodo. He felt like every nerve in his body was hurting. Suddenly, he thought of Sam and his calming smile. Frodo had betrayed him."
d3 = "Frodo’s sword was radiating blue, stronger and stronger every second. Orc were getting closer. And these weren’t just regular orc either, Uruk-Hai were among them. Frodo had killed regular orc before, but he had never stabbed an Uruk-Hai, not with the blue stick."
d4 = "Sam was carrying a small lamp, shedding some blue light. He was afraid that orc might spot him, but it was the only way to avoid deadly pitfalls of Mordor."

docs = [d1.lower(), d2.lower(), d3.lower(), d4.lower()]


ORDERED_VOC = [t.lower() for t in ["Frodo", "Sam", "beast", "orc", "blue"]]
VOC = set(ORDERED_VOC)

defining some functions here that will be needed later on

In [88]:
# stolen from http://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def reduce_to_voc(tokens):
    t_rel = []
    for t in tokens:
        if t in VOC:
            t_rel.append(t)
    return t_rel

def lemmatize(tokens):
    tagged = nltk.pos_tag(tokens)
    for t in tagged:
        try:
            yield LEMMATIZER.lemmatize(t[0], get_wordnet_pos(t[1]))
        except KeyError:
            pass
        
def tokenize(doc):
    doc = " ".join("".join([" " if ch in string.punctuation else ch for ch in doc]).split())
    result = []
    for token in TOKENIZER.tokenize(doc):
        result.append(token.lower())
    return result

def preprocess(doc):
    tokens = tokenize(doc)
    tokens = lemmatize(tokens)
    tokens = reduce_to_voc(tokens)
    return tokens

do preprocessing, the follwoing tokens remain
documents are numbered horizontally, commencing from doc1

In [89]:
docs_prep = []
for doc in docs:
    docs_prep.append(preprocess(doc))
pprint(docs_prep)

[['frodo', 'sam', 'orc', 'sam', 'beast'],
 ['beast', 'frodo', 'sam', 'frodo'],
 ['blue', 'orc', 'orc', 'frodo', 'orc', 'blue'],
 ['sam', 'blue', 'orc']]


compute the idf matrix

In [90]:
def compute_idf(docs):
    mat = np.zeros((len(ORDERED_VOC), len(ORDERED_VOC)))
    no_docs = len(docs)
    for doc in docs:
        for i in range(len(ORDERED_VOC)):
            term = ORDERED_VOC[i]
            mat[i,i] += 1 if term in doc else 0
    return np.divide(mat, no_docs)

idf = compute_idf(docs_prep)
pprint(ORDERED_VOC)
pprint(idf)

['frodo', 'sam', 'beast', 'orc', 'blue']
array([[ 0.75,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.75,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.5 ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.5 ]])


compute the tf matrix

In [91]:
def compute_tf(docs):
    mat = np.zeros((len(docs) ,len(ORDERED_VOC)))
    for d in range(len(docs)):
        doc = docs[d]
        for t in range(len(ORDERED_VOC)):
            term = ORDERED_VOC[t]
            mat[d][t] = doc.count(term)
    return mat

tf = compute_tf(docs_prep)
pprint(tf)

array([[ 1.,  2.,  1.,  1.,  0.],
       [ 2.,  1.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  3.,  2.],
       [ 0.,  1.,  0.,  1.,  1.]])


compute the tf-idf matrix, then transpose it so it is in document-term format

In [92]:
tf_idf = np.matmul(tf, idf)
term_doc_tfidf = np.transpose(tf_idf)
pprint(term_doc_tfidf)

array([[ 0.75,  1.5 ,  0.75,  0.  ],
       [ 1.5 ,  0.75,  0.  ,  0.75],
       [ 0.5 ,  0.5 ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  2.25,  0.75],
       [ 0.  ,  0.  ,  1.  ,  0.5 ]])


### b)

In [93]:
def svd(mat):
    return np.linalg.svd(mat)
    
u, sigma, v = svd(term_doc_tfidf)
sigma = np.diag(sigma)
    
pprint([u, sigma, v])

[array([[-0.46176305, -0.4352475 , -0.73434277,  0.08708796, -0.22470177],
       [-0.40142314, -0.5883274 ,  0.66623669,  0.16253806, -0.14980118],
       [-0.13572251, -0.28374531, -0.05707485, -0.29992719,  0.89880706],
       [-0.72507269,  0.52902442,  0.11562721, -0.41883529, -0.07490059],
       [-0.28545288,  0.3225713 , -0.01556212,  0.83702912,  0.33705265]]),
 array([[ 3.10777963,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.96446704,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.04054244,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.29554427]]),
 array([[-0.50200559, -0.34158573, -0.72823333, -0.31778261],
       [-0.48564371, -0.62917291,  0.60395038,  0.05946061],
       [ 0.48703533, -0.60581294, -0.29422921,  0.55607233],
       [-0.5243445 ,  0.34706097, -0.13545965,  0.76567762]])]


### c)
# unfinished

In [94]:
sigma_2 = sigma[:2][:]
pprint(sigma_2)

dense_vec = np.matmul(sigma_2, np.transpose(v))
pprint(dense_vec)

array([[ 3.10777963,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.96446704,  0.        ,  0.        ]])
array([[-1.56012276, -1.50927363,  1.51359849, -1.62954716],
       [-0.67103391, -1.23598945, -1.19009955,  0.68178984]])


### d)
# unfinished

In [95]:
query= ['sam', 'blue', 'orc']
tf_query = compute_tf([query])
tf_idf_query = np.transpose(np.matmul(tf_query, idf))
pprint(tf_idf_query)

u_2 = u[:][:2]
       
dense_query = np.matmul(np.transpose(u_2), tf_idf_query)
pprint(dense_query)


array([[ 0.  ],
       [ 0.75],
       [ 0.  ],
       [ 0.75],
       [ 0.5 ]])


ValueError: shapes (5,2) and (5,1) not aligned: 2 (dim 1) != 5 (dim 0)

## Exercise 2
### a)

In [125]:
doc_term_idfvector = [
    [0.17, 0.21, 0.35, 0.44, 0.49, 0.39, 0.09, 0.07, 0.37, 0.24],
    [0.49, 0.48, 0.44, 0.09, 0.24, 0.2, 0.41, 0.16, 0.1, 0.15],
    [0.41, 0.36, 0.27, 0.19, 0.15, 0.42, 0.23, 0.42, 0.02, 0.42],
    [0.31, 0.41, 0.21, 0.19, 0.47, 0.28, 0.21, 0.39, 0.16, 0.38],
    [0.46, 0.12, 0.21, 0.25, 0.38, 0.38, 0.46, 0.23, 0.31, 0.14],
    [0.13, 0.33, 0.28, 0.42, 0.07, 0.13, 0.58, 0.15, 0.0, 0.49],
    [0.21, 0.09, 0.07, 0.09, 0.3, 0.54, 0.24, 0.43, 0.51, 0.21],
    [0.18, 0.39, 0.42, 0.05, 0.41, 0.1, 0.52, 0.12, 0.14, 0.38],
    [0.4, 0.51, 0.01, 0.1, 0.12, 0.22, 0.26, 0.34, 0.42, 0.38]
]

def cosine(doc1, doc2):
    length_doc1 = 0
    length_doc2 = 0
    scalar = 0.0
    if not len(doc1) == len(doc2):
        raise ValueError('Vectors of different length')
    for i in range(len(doc1)):
        scalar += doc1[i] * doc2[i]
        length_doc1 += math.pow(doc1[i], 2) 
        length_doc2 += math.pow(doc2[i], 2)
    length_doc1, length_doc2 = math.sqrt(length_doc1), math.sqrt(length_doc2)
    return scalar / (length_doc1 + length_doc2)

def avg_cluster_similarity(cluster, doc):
    similarity = 0.0
    for member in cluster:
        similarity += cosine(member, doc)
    return similarity/len(cluster)
    

def single_pass_clustering(docs, lamda):
    clusters = [[docs[0]]]
    for i in range(1, len(docs)):
        doc = docs[i]
        # this will find the best cluster
        max_avg_similarity = 0.0
        best_cluster = None
        for j in range(len(clusters)):
            cluster = clusters[j]
            cluster_sim = avg_cluster_similarity(cluster, doc)
            if cluster_sim > max_avg_similarity:
                best_cluster = j
                max_avg_similarity = 0.0
        # assign to cluster if greater lamda
        if max_avg_similarity > lamda:
            clusters[best_cluster].append(doc)
        else:
            clusters.append([doc])
    return clusters


def print_clusters(c):
    for i in range(len(c)):
        print('Cluster %s\n\t%s' %(str(i), pprint(c[i])))
    
spc0_6 = single_pass_clustering(doc_term_idfvector, 15000.4)

print_clusters(spc0_6)

[[0.17, 0.21, 0.35, 0.44, 0.49, 0.39, 0.09, 0.07, 0.37, 0.24]]
Cluster 0
	None
[[0.49, 0.48, 0.44, 0.09, 0.24, 0.2, 0.41, 0.16, 0.1, 0.15]]
Cluster 1
	None
[[0.41, 0.36, 0.27, 0.19, 0.15, 0.42, 0.23, 0.42, 0.02, 0.42]]
Cluster 2
	None
[[0.31, 0.41, 0.21, 0.19, 0.47, 0.28, 0.21, 0.39, 0.16, 0.38]]
Cluster 3
	None
[[0.46, 0.12, 0.21, 0.25, 0.38, 0.38, 0.46, 0.23, 0.31, 0.14]]
Cluster 4
	None
[[0.13, 0.33, 0.28, 0.42, 0.07, 0.13, 0.58, 0.15, 0.0, 0.49]]
Cluster 5
	None
[[0.21, 0.09, 0.07, 0.09, 0.3, 0.54, 0.24, 0.43, 0.51, 0.21]]
Cluster 6
	None
[[0.18, 0.39, 0.42, 0.05, 0.41, 0.1, 0.52, 0.12, 0.14, 0.38]]
Cluster 7
	None
[[0.4, 0.51, 0.01, 0.1, 0.12, 0.22, 0.26, 0.34, 0.42, 0.38]]
Cluster 8
	None
